In [1]:
import pandas as pd 
import numpy as np 
import requests
import os
import json
import pypyodbc as odbc #pip install pypyodbc
import io
import zipfile
import chardet



# Database connection details
DRIVER = "SQL Server"
SERVER_NAME = "DESKTOP-FB9GSJB"
DATABASE_NAME = "EnergyConsumption"


** SQL Server Connection String

In [2]:
# Function to create a connection string
def connection_string(driver, server_name, database_name):
    return f"""
        DRIVER={{{driver}}};
        SERVER={server_name};
        DATABASE={database_name};
        Trusted_Connection=yes;
    """

** Url of Datasets

In [3]:
building_url = "https://services2.arcgis.com/11XBiaBYA9Ep0yNJ/arcgis/rest/services/BuildingDetails/FeatureServer/replicafilescache/BuildingDetails_-7832066962313449791.csv"
energy_usage_url = "https://services2.arcgis.com/11XBiaBYA9Ep0yNJ/arcgis/rest/services/HRM_Building_Energy_Usage/FeatureServer/replicafilescache/HRM_Building_Energy_Usage_-3651640421373328302.csv"
fuelOil_url = "https://www150.statcan.gc.ca/n1/tbl/csv/18100001-eng.zip"
electricity_url = "https://www150.statcan.gc.ca/n1/tbl/csv/18100204-eng.zip"
propane_url="https://www2.nrcan.gc.ca/eneene/sources/pripri/prices_byyear_e.cfm?productID=6&downloadXLS"

I. Building Data <br>
1. Download & Get "Building Detail" data

In [4]:
# Function to download the CSV file
def download_BuildingDetails(building_url):  

    # Download CSV
    response = requests.get(building_url)

    if response.status_code == 200:
        file_path = os.path.abspath("Building_Details.csv")
        with open(file_path, "wb") as file:
            file.write(response.content)
        print("CSV file downloaded successfully!")
    else:
        print("Failed to download CSV file.")
        return None

    # Load CSV into DataFrame
    building_detail = pd.read_csv(file_path, dtype={"Building_Name": "str"}, low_memory=False)

    return building_detail

building_data = download_BuildingDetails(building_url)
building_data

CSV file downloaded successfully!


,BUILDING_ID,BUILDING_NAME,NAME_STATUS,NAME_APPROVED_DATE,BUILDING_OWNER,YEAR_OF_CONSTRUCTION,INSTALL_YEAR_CONFIDENCE,TOTAL_SQUARE_FOOTAGE,TOTAL_SQFT_CONFIDENCE,HRM_INTEREST,...,SOURCE,SOURCE_ACCURACY,CIVIC_ID,PID,CIVIC_NUMBER,STREET_NAME,STREET_TYPE,COMMUNITY_NAME,DISTRICT,ObjectId
0,BL10094,NaN,NaN,NaN,PRIV,NaN,NaN,NaN,NaN,NaN,...,FDM PROJ,IN,54825,213975,7,BROOKHOUSE,RD,DARTMOUTH,6.0,1
1,BL100940,NaN,NaN,NaN,PRIV,NaN,NaN,NaN,NaN,NaN,...,HRMCA,IN,12281,615104,37,DOWNIE,DR,HEAD OF ST MARGARETS BAY,13.0,2
2,BL100941,NaN,NaN,NaN,PRIV,NaN,NaN,NaN,NaN,NaN,...,HRMCA,IN,12297,615096,206,MASONS POINT,RD,HEAD OF ST MARGARETS BAY,13.0,3
3,BL100942,NaN,NaN,NaN,PRIV,NaN,NaN,NaN,NaN,NaN,...,HRMCA,IN,12262,40044141,202,MASONS POINT,RD,HEAD OF ST MARGARETS BAY,13.0,4
4,BL100943,NaN,NaN,NaN,PRIV,NaN,NaN,NaN,NaN,NaN,...,HRMCA,IN,12318,615203,201,MASONS POINT,RD,HEAD OF ST MARGARETS BAY,13.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148304,BL99994,NaN,NaN,NaN,PRIV,NaN,NaN,NaN,NaN,NaN,...,HRMCA,IN,106954,40532319,45,TWO RIVERS,DR,MINEVILLE,2.0,148305
148305,BL99995,NaN,NaN,NaN,PRIV,NaN,NaN,NaN,NaN,NaN,...,HRMCA,IN,106981,40579112,49,TWO RIVERS,DR,MINEVILLE,2.0,148306
148306,BL99996,NaN,NaN,NaN,PRIV,NaN,NaN,NaN,NaN,NaN,...,HRMCA,IN,107186,40579120,1,RIVERFRONT,CRT,MINEVILLE,2.0,148307
148307,BL99998,NaN,NaN,NaN,PRIV,NaN,NaN,NaN,NaN,NaN,...,HRMCA,IN,107185,40579138,5,RIVERFRONT,CRT,MINEVILLE,2.0,148308


2. manipultate building data

In [5]:
def menipulate_buildingData(building_data):
    # Remove duplicates
    building_data = building_data.drop_duplicates()

    # Filter for buildings where PRIMARY_USE is "Y"
    if "PRIMARY_USE" in building_data.columns:
        building_data = building_data[building_data["PRIMARY_USE"] == "Y"]

    # Select only relevant columns
    selected_columns = [
        "BUILDING_ID", "BUILDING_NAME", "YEAR_OF_CONSTRUCTION",
        "TOTAL_SQUARE_FOOTAGE", "BUILDING_CLASSIFICATION", "BUILDING_USE"
    ]
    building_data = building_data[selected_columns]

    # Rename columns to match SQL database
    building_data = building_data.rename(columns={
        "BUILDING_ID": "Building_ID",
        "BUILDING_NAME": "Building_Name",
        "YEAR_OF_CONSTRUCTION": "Year_Built",
        "TOTAL_SQUARE_FOOTAGE": "Total_Sqft",
        "BUILDING_CLASSIFICATION": "Building_Classification",
        "BUILDING_USE": "Building_Use"
    })

    # Fill NaN values in "Building_Name" to prevent errors
    building_data["Building_Name"] = building_data["Building_Name"].fillna("Unknown")

    pool_name = ["NEEDHAM COMMUNITY CENTRE", "CANADA GAMES CENTRE" ]
    arena_name = ["RBC CENTRE","EAST COAST VARSITY RINK", "DR GERALD J LEBRUN MEMORIAL CENTRE",
               "HALIFAX FORUM CIVIC CENTRE", "ST MARGARETS CENTRE", "GREENFOOT ENERGY CENTRE"]
    arena_pool = ["ZATZMAN SPORTSPLEX", "SACKVILLE SPORTS STADIUM", "COLE HARBOUR PLACE"]


    # Assign "Facility" column based on Building_Name
    building_data["Facility"] = "Other"  # Default category
    building_data.loc[
        building_data["Building_Name"].str.contains("ARENA", case=False, na=False) |
        building_data["Building_Name"].isin(arena_name),
        "Facility"
    ] = "arena"

    building_data.loc[
        building_data["Building_Name"].str.contains("POOL", case=False, na=False) |
        building_data["Building_Name"].isin(pool_name),
        "Facility"
    ] = "swimming pool"

    building_data.loc[
        building_data["Building_Name"].isin(arena_pool),
        "Facility"
    ] = "arena_pool"

        # Ensure Building_ID is a string and clean it
    building_data["Building_ID"] = building_data["Building_ID"].astype(str).str.strip()
    building_data["Building_ID"] = building_data["Building_ID"].str.replace(r"[^\x00-\x7F]+", "", regex=True)
    building_data = building_data.drop_duplicates(subset=["Building_ID"], keep="first")

    # Clean Building_Name (fix NaN issue)
    building_data["Building_Name"] = building_data["Building_Name"].astype(str).fillna("Unknown")

    # Clean text fields and replace special characters
    building_data["Building_Classification"] = building_data["Building_Classification"].astype(str).fillna("Unknown").str.replace("/", "-", regex=True)
    building_data["Building_Use"] = building_data["Building_Use"].astype(str).fillna("Unknown").str.replace("/", "-", regex=True)
    building_data["Facility"] = building_data["Facility"].astype(str).fillna("Unknown").str.replace("/", "-", regex=True)

    # Convert numerical columns
    building_data["Year_Built"] = pd.to_numeric(building_data["Year_Built"], errors="coerce").fillna(0).astype(int)
    building_data["Total_Sqft"] = pd.to_numeric(building_data["Total_Sqft"], errors="coerce").fillna(0).astype(float)
  

    return building_data


building= menipulate_buildingData(building_data)


3. Store building data in the SQL Server

In [ ]:
# Function to insert/update data into SQL Server
def insert_buildingData_into_sql(building):
    # Select only required columns
    columns = ["Building_ID", "Building_Classification", "Building_Use", "Building_Name", "Year_Built", "Total_Sqft", "Facility"]
    df_data = building[columns]

    # Convert DataFrame to list of tuples for SQL execution
    records = df_data.values.tolist()

    print(f"✅ Total Records to Process: {len(records)}")

    # Connect to SQL Server
    try:
        conn = odbc.connect(connection_string(DRIVER, SERVER_NAME, DATABASE_NAME))
        conn.autocommit = False    
        cursor = conn.cursor()
        cursor.execute("SET LOCK_TIMEOUT 5000;")
        print("\n✅ Connected to SQL Server successfully!")
    except Exception as e:
        print("\n❌ Connection Error:", str(e))
        return

    # Check if the Building table is empty
    try:
        cursor.execute("SELECT COUNT(*) FROM Building;")
        row_count = cursor.fetchone()[0]
        print(f"\n✅ Existing Records in Building Table: {row_count}")
    except Exception as e:
        print("\n❌ Error checking table count:", str(e))
        return

    # Batch size for updates/inserts
    batch_size = 1000  

    if row_count == 0:
        # **Insert all records if table is empty**
        print("\n⚡ Table is empty. Inserting all records.")
        sql_insert = """
            INSERT INTO Building (Building_ID, Building_Classification, Building_Use, Building_Name, Year_Built, Total_Sqft, Facility) 
            VALUES (?, ?, ?, ?, ?, ?, ?)
        """
        try:
            cursor.executemany(sql_insert, records)
            conn.commit()
            print("\n✅ All data inserted successfully!")
        except Exception as e:
            conn.rollback()
            print("\n❌ Error inserting data:", str(e))
    else:
        # **Step 1: Update existing records**
        sql_update = """
            UPDATE Building
            SET Building_Classification = ?, 
                Building_Use = ?, 
                Building_Name = ?, 
                Year_Built = ?, 
                Total_Sqft = ?, 
                Facility = ?
            WHERE Building_ID = ?
        """
        try:
            for i in range(0, len(records), batch_size):
                batch = records[i:i + batch_size]
                cursor.executemany(sql_update, [(b[1], b[2], b[3], b[4], b[5], b[6], b[0]) for b in batch])
                conn.commit()
            print("\n✅ Existing records updated successfully!")
        except Exception as e:
            conn.rollback()
            print("\n❌ Error updating records:", str(e))

        # **Step 2: Insert new records only if they don’t exist**
        sql_insert_new = """
            INSERT INTO Building (Building_ID, Building_Classification, Building_Use, Building_Name, Year_Built, Total_Sqft, Facility)
            SELECT ?, ?, ?, ?, ?, ?, ?
            WHERE NOT EXISTS (
                SELECT 1 FROM Building WHERE Building_ID = ?
            )
        """
        try:
            for i in range(0, len(records), batch_size):
                batch = records[i:i + batch_size]
                cursor.executemany(sql_insert_new, [(b[0], b[1], b[2], b[3], b[4], b[5], b[6], b[0]) for b in batch])
                conn.commit()
            print("\n✅ New records inserted successfully!")
        except Exception as e:
            conn.rollback()
            print("\n❌ Error inserting new records:", str(e))

    # Close connection
    cursor.close()
    conn.close()
    print("\n✅ Connection closed.")

# Call function with a DataFrame
insert_buildingData_into_sql(building)


✅ Total Records to Process: 124282

✅ Connected to SQL Server successfully!

⚡ Merging data into Building table...


II. Energy Table

In [ ]:
def insert_Energy_into_sql():
    
    energy = pd.DataFrame({
        "Energy_ID": [1, 2, 3, 4],
        "Energy_Type": ["electricity", "fuel oil", "natural gas", "propane"]
    })

    # Select only required columns (Ensure column names match SQL table)
    columns = ["Energy_ID", "Energy_Type"]
    df_data = energy[columns]

    
    # Convert DataFrame to list of tuples for SQL execution
    records = df_data.values.tolist()

    print(f"✅ Total Records to Process: {len(records)}")

    # Connect to SQL Server
    try:
        conn = odbc.connect(connection_string(DRIVER, SERVER_NAME, DATABASE_NAME))
        conn.autocommit = True  
        cursor = conn.cursor()
        print("\n✅ Connected to SQL Server successfully!")
    except Exception as e:
        print("\n❌ Connection Error:", str(e))
        return

    # Check if the Energy table is empty
    try:
        cursor.execute("SELECT COUNT(*) FROM Energy;")
        row_count = cursor.fetchone()[0]
        print(f"\n✅ Existing Records in Energy Table: {row_count}")
    except Exception as e:
        print("\n❌ Error checking table count:", str(e))
        return

    if row_count == 0:
        # If table is empty, insert all records
        print("\n⚡ Table is empty. Inserting all records.")
        sql_insert = """
            INSERT INTO Energy (Energy_ID, Energy_Type)
            VALUES (?, ?)
        """
        try:
            cursor.executemany(sql_insert, records)
            conn.commit()
            print("\n✅ All data inserted successfully!")
        except Exception as e:
            conn.rollback()
            print("\n❌ Error inserting data:", str(e))
    else:
        print("\n⚡ Table has data. Updating existing records and inserting new ones.")

        # Step 1: Update existing records
        sql_update = """
            UPDATE Energy
            SET Energy_Type = ?
            WHERE Energy_ID = ?
        """
        try:
            cursor.executemany(sql_update, [(b[1], b[0]) for b in records])
            conn.commit()
            print("\n✅ Existing records updated successfully!")
        except Exception as e:
            conn.rollback()
            print("\n❌ Error updating records:", str(e))

        # Step 2: Insert new records (if they don't already exist)
        sql_insert_new = """
            INSERT INTO Energy (Energy_ID, Energy_Type)
            SELECT ?, ?
            WHERE NOT EXISTS (SELECT 1 FROM Energy WHERE Energy_ID = ?)
        """
        try:
            cursor.executemany(sql_insert_new, [(b[0], b[1], b[0]) for b in records])
            conn.commit()
            print("\n✅ New records inserted successfully!")
        except Exception as e:
            conn.rollback()
            print("\n❌ Error inserting new records:", str(e))

    # Close connection
    cursor.close()
    conn.close()
    print("\n✅ Connection closed.")

insert_Energy_into_sql()


✅ Total Records to Process: 4

✅ Connected to SQL Server successfully!

✅ Existing Records in Energy Table: 4

⚡ Table has data. Updating existing records and inserting new ones.

✅ Existing records updated successfully!

✅ New records inserted successfully!

✅ Connection closed.


III Building_Energy Table </br>
    1. Download Datasets

In [ ]:
def downlad_fuelData(fuelOil_url):

    # Download the ZIP file
    response = requests.get(fuelOil_url)
    zip_file = zipfile.ZipFile(io.BytesIO(response.content))

    # List the files in the ZIP
    zip_file_names = zip_file.namelist()
    print("Files in ZIP:", zip_file_names)

    # Read the main data file (18100001.csv)
    with zip_file.open("18100001.csv") as file:
        fuel_data = pd.read_csv(file)

    return fuel_data

# Save locally (optional)
# df.to_csv("fuel_prices.csv", index=False)
fuel_data = downlad_fuelData(fuelOil_url)

fuel_data


Files in ZIP: ['18100001.csv', '18100001_MetaData.csv']


,REF_DATE,GEO,DGUID,Type of fuel,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,1979-01,"St. John's, Newfoundland and Labrador",2011S0503001,Regular unleaded gasoline at full service fill...,Cents per litre,57,units,0,v735046,2.1,26.0,NaN,NaN,t,1
1,1979-01,"Charlottetown and Summerside, Prince Edward Is...",NaN,Regular unleaded gasoline at full service fill...,Cents per litre,57,units,0,v735056,3.1,24.6,NaN,NaN,t,1
2,1979-01,"Halifax, Nova Scotia",2011S0503205,Regular unleaded gasoline at full service fill...,Cents per litre,57,units,0,v735057,4.1,23.4,NaN,NaN,t,1
3,1979-01,"Saint John, New Brunswick",2011S0503310,Regular unleaded gasoline at full service fill...,Cents per litre,57,units,0,v735058,5.1,23.2,NaN,NaN,t,1
4,1979-01,"Québec, Quebec",2011S0503421,Regular unleaded gasoline at full service fill...,Cents per litre,57,units,0,v735059,6.1,22.6,NaN,NaN,t,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44857,2025-01,"Whitehorse, Yukon",2011A00056001009,Household heating fuel,Cents per litre,57,units,0,v735155,18.7,167.4,NaN,NaN,NaN,1
44858,2025-01,"Yellowknife, Northwest Territories",2011A00056106023,Regular unleaded gasoline at self service fill...,Cents per litre,57,units,0,v735091,19.2,162.6,NaN,NaN,NaN,1
44859,2025-01,"Yellowknife, Northwest Territories",2011A00056106023,Premium unleaded gasoline at self service fill...,Cents per litre,57,units,0,v735109,19.4,184.1,NaN,NaN,NaN,1
44860,2025-01,"Yellowknife, Northwest Territories",2011A00056106023,Diesel fuel at self service filling stations,Cents per litre,57,units,0,v735143,19.6,179.9,NaN,NaN,NaN,1


In [ ]:
def download_electricityData(electricity_url):

    # Download the ZIP file
    response = requests.get(electricity_url)
    zip_file = zipfile.ZipFile(io.BytesIO(response.content))

    # List the files in the ZIP
    zip_file_names = zip_file.namelist()
    print("Files in ZIP:", zip_file_names)

    # Read the main data file (18100204.csv)
    with zip_file.open("18100204.csv") as file:
        electricity_data = pd.read_csv(file)

    return electricity_data

# Save locally (optional)
# df.to_csv("fuel_prices.csv", index=False)
electricity_data = download_electricityData(electricity_url)

electricity_data

Files in ZIP: ['18100204.csv', '18100204_MetaData.csv']


,REF_DATE,GEO,DGUID,Index,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,1981-01,Canada,2016A000011124,"Electric power selling price indexes, national...","Index, 2014=100",351,units,0,v107792869,1.1,31.0,NaN,NaN,NaN,1
1,1981-01,Canada,2016A000011124,Electric power selling price over 5000kw,"Index, 2014=100",351,units,0,v107792870,1.2,31.1,NaN,NaN,NaN,1
2,1981-01,Canada,2016A000011124,Electric power selling price under 5000kw,"Index, 2014=100",351,units,0,v107792871,1.3,30.8,NaN,NaN,NaN,1
3,1981-01,Atlantic Region,2016A00011,Electric power selling price over 5000kw,"Index, 2014=100",351,units,0,v107792873,2.2,38.2,NaN,NaN,NaN,1
4,1981-01,Atlantic Region,2016A00011,Electric power selling price under 5000kw,"Index, 2014=100",351,units,0,v107792874,2.3,45.2,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13723,2024-12,Saskatchewan,2016A000247,Electric power selling price under 5000kw,"Index, 2014=100",351,units,0,v107792901,11.3,127.0,NaN,NaN,NaN,1
13724,2024-12,Alberta,2016A000248,Electric power selling price over 5000kw,"Index, 2014=100",351,units,0,v107792903,12.2,138.8,NaN,NaN,NaN,1
13725,2024-12,Alberta,2016A000248,Electric power selling price under 5000kw,"Index, 2014=100",351,units,0,v107792904,12.3,127.4,NaN,NaN,NaN,1
13726,2024-12,British Columbia,2016A000259,Electric power selling price over 5000kw,"Index, 2014=100",351,units,0,v107792906,13.2,131.7,NaN,NaN,NaN,1


In [ ]:
def mergy_energyData(fuel_data, electricity_data):

    # fuel data
    fuel_data = fuel_data.loc[(fuel_data["GEO"] == "Halifax, Nova Scotia") & 
                              (fuel_data["Type of fuel"] == "Household heating fuel")]
    
    fuel_selected_columns = ["VALUE", "UOM", "REF_DATE"]
    fuel_data = fuel_data[fuel_selected_columns]

    # Rename columns to match SQL database
    fuel_data = fuel_data.rename(columns={
        "VALUE": "Price",
        "UOM": "Measurement_Unit",
        "REF_DATE": "Date",
    })

    fuel_data["Energy_ID"] = 1
    fuel_data["Energy_Type"] = "Fuel Oil"

    # Electricity data
    electricity_data = electricity_data.loc[(electricity_data["GEO"] == "Nova Scotia") & 
                                            (electricity_data["Index"] == "Electric power selling price over 5000kw")]
    electricity_selected_columns = ["VALUE", "UOM", "REF_DATE"]
    electricity_data = electricity_data[electricity_selected_columns]

    # Rename columns to match SQL database
    electricity_data = electricity_data.rename(columns={
        "VALUE": "Price",
        "UOM": "Measurement_Unit",
        "REF_DATE": "Date",
    })

    electricity_data["Energy_ID"] = 1
    electricity_data["Energy_Type"] = "Electricity"

    # Concat two datasets
    energy_data = pd.concat([fuel_data,electricity_data], ignore_index=True)


    # Convert YYYY-MM format to YYYY-MM-01 (first day of the month)
    energy_data["Date"] = pd.to_datetime(energy_data["Date"].astype(str) + "-01", errors="coerce").dt.strftime("%Y-%m-%d")

    return energy_data
 
energy_price = mergy_energyData(fuel_data, electricity_data)
energy_price

,Price,Measurement_Unit,Date,Energy_ID,Energy_Type
0,29.6,Cents per litre,1990-01-01,1,Fuel Oil
1,31.5,Cents per litre,1990-02-01,1,Fuel Oil
2,32.1,Cents per litre,1990-03-01,1,Fuel Oil
3,32.1,Cents per litre,1990-04-01,1,Fuel Oil
4,32.1,Cents per litre,1990-05-01,1,Fuel Oil
...,...,...,...,...,...
944,131.8,"Index, 2014=100",2024-08-01,1,Electricity
945,131.8,"Index, 2014=100",2024-09-01,1,Electricity
946,131.8,"Index, 2014=100",2024-10-01,1,Electricity
947,131.8,"Index, 2014=100",2024-11-01,1,Electricity


In [ ]:
# Function to insert data into SQL Server
def insert_energyPrice_into_sql(energy_price):
    

    # Ensure the column names match the SQL table
    columns = ["Price", "Measurement_Unit", "Date", "Energy_ID"]

    df_data = energy_price[columns]

    # Convert DataFrame to list of tuples for SQL execution
    records = df_data.values.tolist()

    print(f"✅ Total Records to Process: {len(records)}")

    # Connect to SQL Server
    try:
        conn = odbc.connect(connection_string(DRIVER, SERVER_NAME, DATABASE_NAME))
        conn.autocommit = True  
        cursor = conn.cursor()
        print("\n✅ Connected to SQL Server successfully!")
    except Exception as e:
        print("\n❌ Connection Error:", str(e))
        return

    # Check if the Energy_Price table is empty
    try:
        cursor.execute("SELECT COUNT(*) FROM Energy_Price;")
        row_count = cursor.fetchone()[0]
        print(f"\n✅ Existing Records in Energy_Price Table: {row_count}")
    except Exception as e:
        print("\n❌ Error checking table count:", str(e))
        return

    if row_count == 0:
        # If table is empty, insert all records
        print("\n⚡ Table is empty. Inserting all records.")
        sql_insert = """
            INSERT INTO Energy_Price (Price, Measurement_Unit, Date, Energy_ID) 
            VALUES (?, ?, ?, ?)
        """
        try:
            cursor.executemany(sql_insert, records)
            conn.commit()
            print("\n✅ All data inserted successfully!")
        except Exception as e:
            conn.rollback()
            print("\n❌ Error inserting data:", str(e))
    else:
        print("\n⚡ Table has data. Updating existing records and inserting new ones.")

        # Step 1: Update existing records
        sql_update = """
            UPDATE Energy_Price
            SET Price = ?, 
                Measurement_Unit = ?
            WHERE Date = ? AND Energy_ID = ?
        """
        try:
            cursor.executemany(sql_update, [(b[0], b[1], b[2], b[3]) for b in records])
            conn.commit()
            print("\n✅ Existing records updated successfully!")
        except Exception as e:
            conn.rollback()
            print("\n❌ Error updating records:", str(e))

        # Step 2: Insert new records (if they don't already exist)
        sql_insert_new = """
            INSERT INTO Energy_Price (Price, Measurement_Unit, Date, Energy_ID)
            SELECT ?, ?, ?, ?
            WHERE NOT EXISTS (SELECT 1 FROM Energy_Price WHERE Date = ? AND Energy_ID = ?)
        """
        try:
            cursor.executemany(sql_insert_new, [(b[0], b[1], b[2], b[3], b[2], b[3]) for b in records])
            conn.commit()
            print("\n✅ New records inserted successfully!")
        except Exception as e:
            conn.rollback()
            print("\n❌ Error inserting new records:", str(e))

    # Close connection
    cursor.close()
    conn.close()
    print("\n✅ Connection closed.")

# Call the function
insert_energyPrice_into_sql(energy_price)


✅ Total Records to Process: 949

✅ Connected to SQL Server successfully!

✅ Existing Records in Energy_Price Table: 948

⚡ Table has data. Updating existing records and inserting new ones.

✅ Existing records updated successfully!

✅ New records inserted successfully!

✅ Connection closed.


In [ ]:
def download_energyUsageData(energy_usage_url):

    # Send a GET request to the URL
    response = requests.get(energy_usage_url)

    # Check if the request was successful
    if response.status_code == 200:
        # Write the content to a CSV file
        with open("HRM_Building_Energy_Usage.csv", "wb") as file:
            file.write(response.content)
        print("CSV file downloaded successfully!")
    else:
        print("Failed to download CSV file.")

    file_path = r"HRM_Building_Energy_Usage.csv"

    energy_usage = pd.read_csv(file_path)

    return energy_usage

energy_usage = download_energyUsageData(energy_usage_url)
energy_usage

CSV file downloaded successfully!


,Energy ID,Energy Type,Portfolio Manager Property ID,Portfolio Manager Property Name,HRM Building ID,Meter ID,Start Date,End Date,Consumption,Unit of Measure,Cost,ObjectId
0,2010806509,Natural Gas,6303414,Halifax City Hall,BL243,41522813,1/1/2018 12:00:00 AM,2/1/2018 12:00:00 AM,316.16,GJ (Gigajoules),4804.96,1
1,2010806510,Natural Gas,6303414,Halifax City Hall,BL243,41522813,2/1/2018 12:00:00 AM,3/1/2018 12:00:00 AM,281.14,GJ (Gigajoules),4385.44,2
2,2273984240,Natural Gas,6303414,Halifax City Hall,BL243,41522813,3/1/2018 12:00:00 AM,4/1/2018 12:00:00 AM,270.95,GJ (Gigajoules),4156.28,3
3,2273984241,Natural Gas,6303414,Halifax City Hall,BL243,41522813,4/1/2018 12:00:00 AM,5/1/2018 12:00:00 AM,201.76,GJ (Gigajoules),2928.71,4
4,2273984242,Natural Gas,6303414,Halifax City Hall,BL243,41522813,5/1/2018 12:00:00 AM,6/1/2018 12:00:00 AM,116.56,GJ (Gigajoules),1528.73,5
...,...,...,...,...,...,...,...,...,...,...,...,...
28846,8410612543,Electricity,16377864,Hubbards Recreation Centre,BL519,108784554,4/10/2024 12:00:00 AM,4/16/2024 12:00:00 AM,189.00,kWh (kilowatt-hours),33.16,28847
28847,8410612542,Electricity,16377864,Hubbards Recreation Centre,BL519,108784554,4/16/2024 12:00:00 AM,6/11/2024 12:00:00 AM,1769.00,kWh (kilowatt-hours),313.17,28848
28848,8619278812,Electricity,16377864,Hubbards Recreation Centre,BL519,108784554,6/11/2024 12:00:00 AM,8/12/2024 12:00:00 AM,761.00,kWh (kilowatt-hours),134.72,28849
28849,8828755042,Electricity,16377864,Hubbards Recreation Centre,BL519,108784554,8/12/2024 12:00:00 AM,10/10/2024 12:00:00 AM,504.00,kWh (kilowatt-hours),89.22,28850


In [ ]:
def split_multimonth_records(row):
 
    start_date = row["Start Date"]
    end_date = row["End Date"]

    # Ensure dates are in datetime format
    if pd.isna(start_date) or pd.isna(end_date):
        return []  # Skip if dates are missing

    split_records = []
    current_date = start_date

    while current_date < end_date:
        next_month = (current_date + pd.DateOffset(months=1)).replace(day=1)
        last_day = min(next_month - pd.Timedelta(days=1), end_date)
        days_in_month = (last_day - current_date).days + 1
        total_days = (end_date - start_date).days + 1

        consumption_split = row["Consumption"] * (days_in_month / total_days)
        cost_split = row["Cost"] * (days_in_month / total_days)

        split_records.append({
            "Building_ID": row["HRM Building ID"],
            "Energy_ID": row["Energy ID"],
            "Energy_Type": row["Energy Type"],
            "Unit_Of_Measure": row["Unit of Measure"],
            "Date": current_date.to_period("M"),
            "Consumption": consumption_split,
            "Cost": cost_split
        })

        current_date = next_month

    return split_records


# Ensure 'Start_Date' and 'End_Date' are in datetime format
energy_usage["Start Date"] = pd.to_datetime(energy_usage["Start Date"], errors="coerce")
energy_usage["End Date"] = pd.to_datetime(energy_usage["End Date"], errors="coerce")

# Apply function row-wise and flatten the results
split_records_list = energy_usage.apply(split_multimonth_records, axis=1).explode().dropna().tolist()

# Convert to DataFrame
split_records_df = pd.DataFrame(split_records_list)
split_records_df

C:\Users\kkoja\AppData\Local\Temp\ipykernel_62560\2252607330.py:38: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  energy_usage["Start Date"] = pd.to_datetime(energy_usage["Start Date"], errors="coerce")
C:\Users\kkoja\AppData\Local\Temp\ipykernel_62560\2252607330.py:39: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  energy_usage["End Date"] = pd.to_datetime(energy_usage["End Date"], errors="coerce")


,Building_ID,Energy_ID,Energy_Type,Unit_Of_Measure,Date,Consumption,Cost
0,BL243,2010806509,Natural Gas,GJ (Gigajoules),2018-01,306.280000,4654.805000
1,BL243,2010806510,Natural Gas,GJ (Gigajoules),2018-02,271.445517,4234.217931
2,BL243,2273984240,Natural Gas,GJ (Gigajoules),2018-03,262.482812,4026.396250
3,BL243,2273984241,Natural Gas,GJ (Gigajoules),2018-04,195.251613,2834.235484
4,BL243,2273984242,Natural Gas,GJ (Gigajoules),2018-05,112.917500,1480.957187
...,...,...,...,...,...,...,...
52074,BL519,8828755042,Electricity,kWh (kilowatt-hours),2024-09,252.000000,44.610000
52075,BL519,8828755042,Electricity,kWh (kilowatt-hours),2024-10,84.000000,14.870000
52076,BL519,9013455368,Electricity,kWh (kilowatt-hours),2024-10,368.322581,65.205161
52077,BL519,9013455368,Electricity,kWh (kilowatt-hours),2024-11,502.258065,88.916129


In [ ]:
def menipulate_energyData(split_records_df):

     # Select only relevant columns
    selected_columns = ["Energy_Type", "Unit_Of_Measure", "Date", "Consumption", "Cost", "Building_ID"]
    energy_usage_data = split_records_df[selected_columns]

    # Map Energy Type to Energy_ID
    energy_id_mapping = {
        "Electricity": 1,
        "Fuel Oil": 2,
        "Natural Gas": 3,
        "Propane": 4
    }
    energy_usage_data["Energy_ID"] = energy_usage_data["Energy_Type"].map(energy_id_mapping)

    # Convert Energy_ID to Integer (Handling NaN)
    energy_usage_data["Energy_ID"] = energy_usage_data["Energy_ID"].fillna(0).astype(int)

    # Drop the original "Energy_Type" column
    energy_usage_data = energy_usage_data.drop(columns=["Energy_Type"])

    # Aggregate data: Sum Consumption and Cost for same Year-Month, Building_ID, and Energy_ID
    energy_usage_data = energy_usage_data.groupby(
        ["Building_ID", "Energy_ID", "Date", "Unit_Of_Measure"]
    ).agg({
        "Consumption": "sum",
        "Cost": "sum"
    }).reset_index()

    return energy_usage_data




building_energy= menipulate_energyData(split_records_df)

building_energy

,Building_ID,Energy_ID,Date,Unit_Of_Measure,Consumption,Cost
0,BL100,1,2015-12,kWh (kilowatt-hours),226.678571,33.548571
1,BL100,1,2016-01,kWh (kilowatt-hours),7234.545567,1070.716601
2,BL100,1,2016-02,kWh (kilowatt-hours),6462.500000,956.450000
3,BL100,1,2016-03,kWh (kilowatt-hours),6822.122016,1009.674058
4,BL100,1,2016-04,kWh (kilowatt-hours),5526.923077,817.984615
...,...,...,...,...,...,...
24152,BL975,1,2024-09,kWh (kilowatt-hours),7482.932540,1089.233492
24153,BL975,1,2024-10,kWh (kilowatt-hours),9058.672222,1267.472222
24154,BL975,1,2024-11,kWh (kilowatt-hours),12072.716667,1708.523125
24155,BL975,1,2024-12,kWh (kilowatt-hours),17192.250000,2382.985312


In [ ]:
import pandas as pd
import pyodbc as odbc  # Ensure you have pyodbc installed

import pandas as pd
import pyodbc as odbc  # Ensure you have pyodbc installed

def insert_EnergyUsage_into_sql(building_energy):

    # ✅ Ensure Building_ID is a string and clean it
    building_energy["Building_ID"] = building_energy["Building_ID"].astype(str).str.strip()
    building_energy["Building_ID"] = building_energy["Building_ID"].str.replace(r"[^\x00-\x7F]+", "", regex=True)

    # ✅ Ensure Date is formatted as YYYY-MM-DD
    building_energy["Date"] = pd.to_datetime(building_energy["Date"].astype(str) + "-01", errors="coerce").dt.strftime("%Y-%m-%d")

    # ✅ Ensure Energy_ID is valid
    valid_energy_ids = {1, 2, 3, 4}  # Adjust based on the Energy table
    building_energy = building_energy[building_energy["Energy_ID"].isin(valid_energy_ids)]

    # ✅ Select only required columns
    columns = ["Unit_Of_Measure", "Consumption", "Cost", "Date", "Building_ID", "Energy_ID"]
    df_data = building_energy[columns].copy()

    # ✅ Ensure Consumption and Cost are numeric
    df_data["Consumption"] = pd.to_numeric(df_data["Consumption"], errors="coerce").fillna(0)
    df_data["Cost"] = pd.to_numeric(df_data["Cost"], errors="coerce").fillna(0)

    # ✅ Convert DataFrame to list of tuples for SQL execution
    records = df_data.values.tolist()

    if not records:
        print("\n⚠️ No valid energy usage data available. Skipping SQL insertion.")
        return

    print(f"✅ Total Records to Process: {len(records)}")

    # ✅ Connect to SQL Server
    try:
        conn = odbc.connect(connection_string(DRIVER, SERVER_NAME, DATABASE_NAME))
        conn.autocommit = False  
        cursor = conn.cursor()
        cursor.execute("SET LOCK_TIMEOUT 5000;")
        print("\n✅ Connected to SQL Server successfully!")
    except Exception as e:
        print("\n❌ Connection Error:", str(e))
        return

    # ✅ Check if Building Table is Empty
    try:
        cursor.execute("SELECT COUNT(*) FROM Building;")
        building_count = cursor.fetchone()[0]
        if building_count == 0:
            print("\n⚠️ The 'Building' table is empty. Inserting default buildings.")
            
            sql_insert_buildings = """
                INSERT INTO Building_Energy (Unit_Of_Measure, Consumption, Cost, Date, Building_ID, Energy_ID)
                VALUES (?, ?, ?, ?, ?, ?)
            """
            cursor.executemany(sql_insert_buildings, records)
            conn.commit()
            print("\n✅ Default buildings inserted successfully.")
    except Exception as e:
        conn.rollback()
        print("\n❌ Error inserting default buildings:", str(e))
        return

    # # ✅ Get Existing Building_IDs from Building Table
    # try:
    #     cursor.execute("SELECT Building_ID FROM Building;")
    #     existing_building_ids = {row[0] for row in cursor.fetchall()}
    #     print(f"\n✅ Found {len(existing_building_ids)} valid Building_IDs in the database.")
    # except Exception as e:
    #     print("\n❌ Error fetching Building_IDs:", str(e))
    #     return

    # # ✅ Filter out invalid Building_IDs
    # valid_records = [b for b in records if b[4] in existing_building_ids]

    # if not valid_records:
    #     print("\n⚠️ No valid Building_IDs found. Skipping SQL insertion.")
    #     return

    # ✅ Batch size for updates/inserts
    batch_size = 1000  

    # ✅ Step 1: Update existing records
    sql_update = """
        UPDATE Building_Energy
        SET Unit_Of_Measure = ?, 
            Consumption = ?, 
            Cost = ?
        WHERE Date = ? AND Building_ID = ? AND Energy_ID = ?
    """
    try:
        for i in range(0, len(records), batch_size):
            batch = records[i:i + batch_size]
            cursor.executemany(sql_update, [(b[0], b[1], b[2], b[3], b[4], b[5]) for b in records])
            conn.commit()
        print("\n✅ Existing records updated successfully!")
    except Exception as e:
        conn.rollback()
        print("\n❌ Error updating records:", str(e))

    # ✅ Step 2: Insert new records only if they don’t exist
    sql_insert_new = """
        INSERT INTO Building_Energy (Unit_Of_Measure, Consumption, Cost, Date, Building_ID, Energy_ID)
        SELECT ?, ?, ?, ?, ?, ?
        WHERE NOT EXISTS (
            SELECT 1 FROM Building_Energy 
            WHERE Date = ? AND Building_ID = ? AND Energy_ID = ?
        )
    """
    try:
        for i in range(0, len(records), batch_size):
            batch = records[i:i + batch_size]
            cursor.executemany(sql_insert_new, [(b[0], b[1], b[2], b[3], b[4], b[5], b[3], b[4], b[5]) for b in records])
            conn.commit()
        print("\n✅ New records inserted successfully!")
    except Exception as e:
        conn.rollback()
        print("\n❌ Error inserting new records:", str(e))

    # ✅ Close connection
    cursor.close()
    conn.close()
    print("\n✅ Connection closed.")

# Call the function
insert_EnergyUsage_into_sql(building_energy)


NameError: name 'building_energy' is not defined